# Agente de Soporte Técnico con Funciones Personalizadas

## Descripción

En este ejercicio, desarrollarás un **agente de soporte técnico de TI** que utiliza funciones personalizadas como herramientas. El agente recopilará información detallada sobre problemas técnicos (errores de software, incidencias de red, problemas de hardware, acceso a sistemas) y generará automáticamente tickets de soporte.

### Características Principales
- **Recopilación inteligente**: El agente solicita información estructurada:
  - Correo corporativo del reportante
  - Sistema o aplicación afectada
  - Ubicación de la oficina (Madrid, Barcelona, Valencia, Sevilla, Bilbao)
  - Severidad del problema e impacto en operaciones
  
- **Generación de tickets**: Crea archivos de ticket con número único
- **Funciones personalizadas**: Aprenderás a integrar lógica personalizada en agentes

### Herramientas Utilizadas
- **FunctionTool**: Permite que el agente invoque funciones Python
- **Auto Function Calling**: El agente decide automáticamente cuándo usar las funciones

### Definir una función personalizada

Primero, definimos la función `submit_support_ticket` que el agente utilizará. Esta función genera un número de ticket, guarda los detalles en un archivo de texto y devuelve un mensaje de confirmación.

In [ ]:
import json
from pathlib import Path
import uuid
from typing import Any, Callable, Set

def submit_support_ticket(email_address: str, description: str) -> str:
    # Use current notebook directory
    script_dir = Path.cwd()
    ticket_number = str(uuid.uuid4()).replace('-', '')[:6]
    file_name = f"ticket-{ticket_number}.txt"
    file_path = script_dir / file_name
    text = (
        f"Ticket de soporte: {ticket_number}\n"
        f"Enviado por: {email_address}\n"
        f"Descripción:\n{description}"
    )
    file_path.write_text(text)

    message_json = json.dumps({
        "message": f"Ticket de soporte {ticket_number} enviado. El archivo del ticket se guardó como {file_name}"
    })
    return message_json

# Define set of functions that agent can call
user_functions: Set[Callable[..., Any]] = {
    submit_support_ticket,
}
print("Function 'submit_support_ticket' defined")

### Cargar librerías y variables de entorno

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# Import required classes from SDK
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder, MessageRole

# Load environment variables
load_dotenv(find_dotenv(usecwd=True))
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")

### Conectar al Cliente de Agentes (Agent Client)

In [ ]:
print("Connecting to Azure AI Agents Client...")
agent_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True
    )
)
print("Agent client connected successfully")

### Definir un agente que puede usar las funciones personalizadas

Creamos un `ToolSet` con nuestra función personalizada y luego creamos un agente con instrucciones sobre cómo usarla.

In [ ]:
print("Setting up function tools...")
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
agent_client.enable_auto_function_calls(toolset)
print("Function tools configured")

print("Creating agent...")
agent = agent_client.create_agent(
    model=model_deployment,
    name="technical-support-agent",
    instructions="""
    Eres un agente de soporte técnico de TI de la empresa.
    Cuando un usuario reporte un problema técnico (errores de software, incidencias de red, problemas de hardware, acceso a sistemas o dudas técnicas), recopila su correo corporativo y una descripción detallada que incluya:
    - Sistema o aplicación afectada (p. ej., ERP, CRM, correo electrónico, VPN, impresoras)
    - Ubicación/oficina (Madrid, Barcelona, Valencia, Sevilla, Bilbao)
    - Severidad del problema e impacto en las operaciones
    Luego envía un ticket de soporte usando la función disponible.
    Si se guarda un archivo, informa al usuario el nombre del archivo y el número de ticket.
    """,
    toolset=toolset,
)
print(f"Agent created: {agent.name} (id: {agent.id})")

print("Creating thread...")
thread = agent_client.threads.create()
print(f"Thread created (id: {thread.id})")

### Enviar un prompt al agente y procesar la respuesta

Ahora puedes interactuar con el agente. El agente te pedirá la información necesaria para crear el ticket de soporte.

In [ ]:
# Modify this prompt with your problem
user_prompt = "Tengo un problema urgente con el acceso al sistema ERP en nuestra oficina de Madrid."

print(f"Sending prompt: '{user_prompt}'")
message = agent_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_prompt,
)
print(f"User message created (thread: {thread.id})")

print("Processing run...")
run = agent_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run completed with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    print("Retrieving agent response...")
    last_msg = agent_client.messages.get_last_message_text_by_role(
        thread_id=thread.id,
        role=MessageRole.AGENT,
    )
    if last_msg:
        print("Agent response:")
        print(f"{last_msg.text.value}")

### Continuar la conversación

Proporciona la información que el agente te solicita. Ejecuta la siguiente celda después de cada respuesta del agente.

In [ ]:
# Reply to agent here
user_prompt = "Mi correo es soporte.ti@example.com. Varios usuarios no pueden acceder al sistema ERP desde esta mañana. Aparece error de autenticación. Afecta a aproximadamente 20 usuarios del departamento de ventas. No tengo más información."

print(f"Sending prompt: '{user_prompt}'")
message = agent_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_prompt,
)
print(f"User message created (thread: {thread.id})")

print("Processing run...")
run = agent_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run completed with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    print("Retrieving agent response...")
    last_msg = agent_client.messages.get_last_message_text_by_role(
        thread_id=thread.id,
        role=MessageRole.AGENT,
    )
    if last_msg:
        print("Agent response:")
        print(f"{last_msg.text.value}")

### Obtener el historial de la conversación

In [ ]:
print("\nConversation history:\n")
messages = agent_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    if message.text_messages:
        last_msg = message.text_messages[-1]
        print(f"{message.role}: {last_msg.text.value}\n")

### Limpieza

In [ ]:
print(f"Deleting agent (id: {agent.id})...")
agent_client.delete_agent(agent.id)
print("Agent deleted successfully")